In [2]:
import os
import warnings
import logging
# Suppress TensorFlow logs
# Suppress warnings
warnings.filterwarnings('ignore')
# Suppress logging messages
logging.getLogger('tensorflow').setLevel(logging.ERROR)
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd 
import numpy as np
import joblib 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder,StandardScaler 


2024-08-23 14:05:47.361179: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-23 14:05:47.518317: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-23 14:05:47.518335: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-23 14:05:47.546679: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-23 14:05:48.172633: W tensorflow/stream_executor/platform/de

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
 
# Load data
# df = pd.read_csv("/content/dqp_edge_metric_data_till_31_12_2027.csv")
df = pd.read_csv("dataset/dqp_edge_metric_data_till_31_12_2027.csv")
df = df.dropna()
df 

,timestamp,CPU utilization@util,CPU load@load1,CPU load@load5,CPU load@load15,Memory@mem_used_percent
0,03-11-2023 00:00,28.77810,4.01800,2.98250,2.28150,83.8479
1,03-11-2023 06:00,90.73230,76.23300,63.69200,42.43170,89.8218
2,03-11-2023 12:00,91.62920,74.75800,68.16150,59.83700,65.2126
3,03-11-2023 18:00,8.54688,2.74917,2.01867,1.51000,52.2027
4,04-11-2023 00:00,9.33542,2.62667,1.87300,1.43283,52.2296
...,...,...,...,...,...,...
7503,30-12-2027 18:00,28.95000,6.13500,4.94917,4.65900,88.5298
7504,31-12-2027 00:00,18.52500,3.48567,2.18283,1.26583,93.5433
7505,31-12-2027 06:00,60.16670,13.79480,11.81680,9.61400,95.5605
7506,31-12-2027 12:00,31.49370,6.56550,5.33067,4.91767,73.6354


In [ ]:

 
# Prepare data for multiple outputs
columns_to_predict = ['CPU load@load1', 'CPU load@load5', 'CPU load@load15', 'CPU utilization@util', 'Memory@mem_used_percent']
timeseries_data = df[columns_to_predict].values
 
# Prepare data function
def prepare_data(timeseries_data, n_features):
    X, y = [], []
    for i in range(len(timeseries_data)):
        end_ix = i + n_features
        if end_ix > len(timeseries_data)-1:
            break
        seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
 
# Define input sequence
n_steps = 3
X, y = prepare_data(timeseries_data, n_steps)
 
# Reshape input data
n_features = len(columns_to_predict)
X = X.reshape((X.shape[0], X.shape[1], n_features))
 
# Define model architecture for multiple outputs
model = Sequential()
model.add(LSTM(500, return_sequences=True, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dropout(0.2))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(LSTM(500, activation='relu', return_sequences=True))
model.add(LSTM(150, return_sequences=True, activation='relu'))
model.add(LSTM(50, activation='relu'))
model.add(Dense(50))
model.add(Dense(50))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(len(columns_to_predict)))  # Output layer with multiple neurons corresponding to each output column
 
# Compile model
model.compile(optimizer='adam', loss='mse')
 
# Fit model
model.fit(X, y, epochs=150)
 
# Future prediction
future_steps = 150  # Number of steps to predict into the future
x_input = timeseries_data[-n_steps:]  # Use the last n_steps as input
 
predictions = []
for _ in range(future_steps):
    x_input = x_input.reshape((1, n_steps, n_features))
    yhat = model.predict(x_input, verbose=0)
    predictions.append(yhat[0])
    x_input = np.concatenate([x_input[:, 1:, :], yhat.reshape((1, 1, n_features))], axis=1)
 
# Inverse transform the predictions (if needed)
# scaler.inverse_transform(predictions)
 
# Print future predictions
print("Future Predictions:")
print(predictions)
import matplotlib.pyplot as plt
 
# Number of data points in the original dataset
num_data_points = len(df)
 
# Generate time indices for future predictions
future_time_indices = np.arange(num_data_points, num_data_points + future_steps)
 
# Plot future predictions separately for each metric
fig, axs = plt.subplots(len(columns_to_predict), 1, figsize=(10, 6*len(columns_to_predict)))
 
for i, column in enumerate(columns_to_predict):
    # Plot real values
    axs[i].plot(df.index, df[column], label='Real Values', color='blue')
    # Plot predicted values
    axs[i].plot(future_time_indices, [prediction[i] for prediction in predictions], label='Predicted Values', color='orange')
    axs[i].set_xlabel('Time Index')
    axs[i].set_ylabel(column)
    axs[i].legend()
 
plt.tight_layout()
plt.show()

In [3]:
# Define DataCleaning class
class DataCleaning(BaseEstimator, TransformerMixin):

    def __init__(self):
        # Prepare data for multiple outputs
        self.columns_to_predict = ['CPU load@load1', 'CPU load@load5', 'CPU load@load15', 'CPU utilization@util', 
                                    'Memory@mem_used_percent']
        self.n_features = 8
        
    # Prepare data function
    def prepare_data(self, timeseries_data, n_features):
        print('Data is preparing...')
        X, y = [], []
        for i in range(len(timeseries_data)):
            end_ix = i + n_features
            if end_ix > len(timeseries_data) - 1:
                break
            seq_x, seq_y = timeseries_data[i:end_ix], timeseries_data[end_ix]
            X.append(seq_x)
            y.append(seq_y)
        
        print('Data preparing done...')
        
        return np.array(X), np.array(y)

    # Define a function to create the LSTM model
    def create_lstm_model(self, units=50, optimizer='adam', hidden_layers=1, hidden_units=50,
                        dropout_rate=0.2, input_shape=(8, 5), learning_rate=0.001):
        model = Sequential()
        model.add(LSTM(units, return_sequences=True, activation='relu', input_shape=input_shape))
        model.add(LSTM(units, activation='relu'))
        
        # Add hidden layers with dropout and batch normalization
        for _ in range(hidden_layers):
            model.add(Dense(hidden_units, activation='relu'))
            model.add(Dropout(dropout_rate))
            model.add(BatchNormalization())
        
        model.add(Dense(len(self.columns_to_predict)))
        
        if optimizer == 'adam':
            opt = Adam(learning_rate=learning_rate)
        else:
            opt = RMSprop(learning_rate=learning_rate)
        
        model.compile(loss='mse', optimizer=opt)
        return model

    # def create_lstm_model(units=70, optimizer='adam', hidden_layers=0, hidden_units=70):
    #     # Define model architecture for multiple outputs
    #     model = Sequential()
    #     model.add(LSTM(units,return_sequences=True, activation='relu',input_shape=(X.shape[1], X.shape[2])))
    #     model.add(LSTM(units, activation='relu',return_sequences=True))
    #     model.add(LSTM(units, activation='relu',return_sequences=True))
    #     model.add(LSTM(units, activation='relu'))

    #     # Add hidden layers
    #     for _ in range(hidden_layers):
    #         model.add(Dense(hidden_units, activation='relu'))

    #     model.add(Dense(len(columns_to_predict)))
    #     model.compile(loss='mse', optimizer=optimizer)
    #     return model


    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Reshape input data
        n_features = len(self.columns_to_predict)
        X = X.reshape((X.shape[0], X.shape[1], n_features))
        return X



In [ ]:
def create_lstm_model(units=70, optimizer='adam', hidden_layers=0, hidden_units=70):
            # Define model architecture for multiple outputs
            model = Sequential()
            model.add(LSTM(units,return_sequences=True, activation='relu',input_shape=(X.shape[1], X.shape[2])))
            model.add(LSTM(units, activation='relu',return_sequences=True))
            model.add(LSTM(units, activation='relu',return_sequences=True))
            model.add(LSTM(units, activation='relu'))

            # Add hidden layers
            for _ in range(hidden_layers):
                model.add(Dense(hidden_units, activation='relu'))

            model.add(Dense(len(columns_to_predict)))
            model.compile(loss='mse', optimizer=optimizer)
            return model
    

In [ ]:
n_steps = 8
    
def cleaning(timeseries_data):
    data_cleaning_obj = DataCleaning()
    X,y = data_cleaning_obj.prepare_data(timeseries_data, n_steps)
    return X,y 

@st.cache
def future_prediction(df,best_model_lr,timeseries_data):
    # Future prediction
    n_features = 5
    future_steps = 500  # Number of steps to predict into the future
    x_input = timeseries_data[-n_steps:]  # Use the last n_steps as input
    
    predictions = []
    for _ in range(future_steps):
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = best_model_lr.predict(x_input)
        predictions.append(yhat)
        x_input = np.concatenate([x_input[:, 1:, :], yhat.reshape((1, 1, n_features))], axis=1)
    
    # Number of data points in the original dataset
    num_data_points = len(df)
    
    # Generate time indices for future predictions
    future_time_indices = np.arange(num_data_points, num_data_points + future_steps)

    return predictions,future_time_indices
    